In [1]:
# imports 
import pandas as pd # to read the csv files 
import os.path
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# need to allow for the memory limit to be able to grow (?) https://www.tensorflow.org/guide/gpu
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [3]:
# maybe we should use IMG instead...
# thanks so much to Alex Kyllo for pointing this out to us!
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
rescale_datagen = ImageDataGenerator(rescale=1./255)
train_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/train',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)
validation_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/valid',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)

Found 99999 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [4]:
root_logdir = os.path.join(os.curdir, 'my_logs')

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [5]:
densenet121 = tf.keras.applications.DenseNet121(include_top=True, 
                                                weights=None, 
                                                input_shape=(256,256,3),
                                               classes=2)

In [6]:
densenet121.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                   metrics=['accuracy'])
densenet121_history = densenet121.fit(train_generator, 
                                  epochs=50, 
                                  validation_data=validation_generator, 
                                  callbacks=[
                                      tf.keras.callbacks.ModelCheckpoint('models/densenet121', save_best_only=True),
                                      tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True),
                                      tf.keras.callbacks.TensorBoard(get_run_logdir())
                                  ])

Epoch 1/50
3125/3125 [==============================] - ETA: 0s - loss: 0.4598 - accuracy: 0.7744WARNING:tensorflow:From C:\Users\valen\anaconda3\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/densenet121\assets
3125/3125 [==============================] - 999s 320ms/step - loss: 0.4598 - accuracy: 0.7744 - val_loss: 0.3892 - val_accuracy: 0.8235
Epoch 2/50
3125/3125 [==============================] - 794s 254ms/step - loss: 0.2080 - accuracy: 0.9150 - val_loss: 0.7187 - val_accuracy: 0.7118
Epoch 3/50
3125/3125 [==============================] - 826s 264ms/step - loss: 0.1159 - accuracy: 0.9548 - val_loss: 0.1899 - val_accuracy: 0.9218
Epoch 4/50
3125/3125 [==========================

In [7]:
test_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/test',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)

Found 20000 images belonging to 2 classes.


In [8]:
densenet121.evaluate(test_generator, batch_size=32)

625/625 [==============================] - 146s 234ms/step - loss: 0.0335 - accuracy: 0.9895


[0.03354091942310333, 0.9894999861717224]